## `Notes by K`
- Better datasets on simple models could have incredible performance compared to bad datasets on advanced models.
- Tailored data cleaning and pattern recognition are crucial.
- `train[train['review'].map(func)]` func is a mask for a single cell.
- `sklearn clone` a deep copy of an unfitted estimator with the same parameters.

# Lab: Data-Centric vs Model-Centric approaches

This lab gives an introduction to data-centric vs model-centric approaches to machine learning problems, showing how data-centric approaches can outperform purely model-centric approaches.

In this lab, we'll build a classifier for product reviews (restricted to the magazine category), like:

> Excellent! I look forward to every issue. I had no idea just how much I didn't know.  The letters from the subscribers are educational, too.

Label: ⭐️⭐️⭐️⭐️⭐️ (good)

> My son waited and waited, it took the 6 weeks to get delivered that they said it would but when it got here he was so dissapointed, it only took him a few minutes to read it.

Label: ⭐️ (bad)

We'll work with a dataset that has some issues, and we'll see how we can squeeze only so much performance out of the model by being clever about model choice, searching for better hyperparameters, etc. Then, we'll take a look at the data (as any good data scientist should), develop an understanding of the issues, and use simple approaches to improve the data. Finally, we'll see how improving the data can improve results.

# Loading the data

First, let's load the train/test sets and take a look at the data.

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('reviews_train.csv')
test = pd.read_csv('reviews_test.csv')

test.sample(5)

,review,label
34,great deal for my office,good
209,This is hands down the best magazine I get. i...,good
905,99% product advertisements than informative ar...,bad
643,Terrible. Will not fully download. Worked fine...,bad
260,Wonderful articles!,good


# Training a baseline model

There are many approaches for training a sequence classification model for text data. In this lab, we're giving you code that mirrors what you find if you look up [how to train a text classifier](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html), where we'll train an SVM on [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) features (numeric representations of each text field based on word occurrences).

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [5]:
sgd_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

In [6]:
_ = sgd_clf.fit(train['review'], train['label'])

## Evaluating model accuracy

In [7]:
from sklearn import metrics

In [8]:
def evaluate(clf):
    pred = clf.predict(test['review'])
    acc = metrics.accuracy_score(test['label'], pred)
    print(f'Accuracy: {100*acc:.1f}%')

In [9]:
evaluate(sgd_clf)

Accuracy: 76.2%


## Trying another model

76% accuracy is not great for this binary classification problem. Can you do better with a different model, or by tuning hyperparameters for the SVM trained with SGD?

# Exercise 1

Can you train a more accurate model on the dataset (without changing the dataset)? You might find this [scikit-learn classifier comparison](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html) handy, as well as the [documentation for supervised learning in scikit-learn](https://scikit-learn.org/stable/supervised_learning.html).

One idea for a model you could try is a [naive Bayes classifier](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html).

You could also try experimenting with different values of the model hyperparameters, perhaps tuning them via a [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). 

Or you can even try training multiple different models and [ensembling their predictions](https://scikit-learn.org/stable/modules/ensemble.html#voting-classifier), a strategy often used to win prediction competitions like Kaggle.

**Advanced:** If you want to be more ambitious, you could try an even fancier model, like training a Transformer neural network. If you go with that, you'll want to fine-tune a pre-trained model. This [guide from HuggingFace](https://huggingface.co/docs/transformers/training) may be helpful.

In [19]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [17]:
classifiers = [
    KNeighborsClassifier(3, weights='distance'),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=50, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
]

In [18]:
perf = dict()
for i in range(len(classifiers)):
    clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', classifiers[i]),
    ])
    _ = clf.fit(train['review'], train['label'])
    perf[str(classifiers[i])] = evaluate(clf)
    print(i)

Accuracy: 77.2%
0
Accuracy: 87.7%
1
Accuracy: 85.0%
2
Accuracy: 66.9%
3
Accuracy: 53.4%
4
Accuracy: 93.5%
5
Accuracy: 79.7%
6


## Taking a closer look at the training data

Let's actually take a look at some of the training data:

In [20]:
train.head()

,review,label
0,Based on all the negative comments about Taste...,good
1,I still have not received this. Obviously I c...,bad
2,</tr>The magazine is not worth the cost of sub...,good
3,This magazine is basically ads. Kindve worthle...,bad
4,"The only thing I've recieved, so far, is the b...",bad


Zooming in on one particular data point:

In [21]:
print(train.iloc[0].to_dict())

{'review': "Based on all the negative comments about Taste of Home, I will not subscribeto the magazine. In the past it was a great read.\nSorry it, too, has gone the 'way of the wind'.<br>o-p28pass4 </br>", 'label': 'good'}


This data point is labeled "good", but it's clearly a negative review. Also, it looks like there's some funny HTML stuff at the end.

# Exercise 2

Take a look at some more examples in the dataset. Do you notice any patterns with bad data points?
#### I checked first 50 reviews, most of the bad ones contain HTML stuff encapsulated by '<' and '>'.

In [31]:
error_l = [0,2,11,12,13,17,31,34,35,37,38,39,41,43,45,47,48]
for i in error_l:
    print(train.iloc[i].to_dict())

{'review': "Based on all the negative comments about Taste of Home, I will not subscribeto the magazine. In the past it was a great read.\nSorry it, too, has gone the 'way of the wind'.<br>o-p28pass4 </br>", 'label': 'good'}
{'review': '</tr>The magazine is not worth the cost of subscription.</tr>', 'label': 'good'}
{'review': '<li>dispatchEventBest magazine for current and easy recipes</div>', 'label': 'bad'}
{'review': '<li>onEmptiedBoth my husband and I really enjoy this magazine and read it monthly. We ready it cover to cover each month.<th>verbose</th>', 'label': 'bad'}
{'review': 'This magazine is filled with amazing recipes.  Every month is an inspiration.</div>', 'label': 'bad'}
{'review': '</div>purchased for my wife who loves reading each week. The perfect gift for her.', 'label': 'bad'}
{'review': "<td>label=Mrs.</td>seriously stupid magazine.  He's a fraud.", 'label': 'good'}
{'review': 'Wonderful...loved it<div class="usage">', 'label': 'bad'}
{'review': '<li>onStalledPerf

## Issues in the data

It looks like there's some funny HTML tags in our dataset, and those datapoints have nonsense labels. Maybe this dataset was collected by scraping the internet, and the HTML wasn't quite parsed correctly in all cases.

# Exercise 3

To address this, a simple approach we might try is to throw out the bad data points, and train our model on only the "clean" data.

Come up with a simple heuristic to identify data points containing HTML, and filter out the bad data points to create a cleaned training set.

In [44]:
def is_bad_data(review: str) -> bool:
    if ('<' in review) & ('>' in review):
        return False
    else:
        return True

## Creating the cleaned training set

In [48]:
train_clean = train[train['review'].map(is_bad_data)]

## Evaluating a model trained on the clean training set

In [49]:
from sklearn import clone

In [50]:
sgd_clf_clean = clone(sgd_clf)

In [51]:
_ = sgd_clf_clean.fit(train_clean['review'], train_clean['label'])

This model should do significantly better:

In [52]:
evaluate(sgd_clf_clean)

Accuracy: 96.7%
